In [179]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("../project1-partA/project1_data.csv")
df.head()

,Date,PX_LAST,PX_OPEN,PX_HIGH,PX_LOW,PX_VOLUME,CUR_MKT_CAP,Ticker,Market_Value
0,2014-01-02,76.27,76.04,76.500,75.90,5212460.0,134027.7199,DIS,1.022229e+07
1,2014-01-03,76.11,76.30,76.697,76.01,4061042.0,133746.5552,DIS,1.017945e+07
2,2014-01-06,75.82,76.53,76.840,75.49,6816693.0,133236.9441,DIS,1.010203e+07
3,2014-01-07,76.34,76.12,76.700,75.90,4511157.0,134150.7295,DIS,1.024107e+07
4,2014-01-08,75.22,76.13,76.280,74.81,10914858.0,132182.5763,DIS,9.942773e+06


In [180]:
n_tickers = len(df["Ticker"].unique())

df['Date'] = pd.to_datetime(df['Date'])

# Column called total market value that is the total market value per day
df["Total_Market_Value_Day"] = df.groupby('Date')['Market_Value'].transform("sum")

# Value Weight column
df["Value_Weight"] = df["Market_Value"] / df["Total_Market_Value_Day"]

# Equal Weight column
df["Equal_Weight"] = 1 / n_tickers


In [181]:
import time
from datetime import datetime
specific_date = "2014-03-06"

def pie_chart_for_specific_date(df,date):
    df_for_pie = df[df["Date"]==date]
    fig = px.pie(df_for_pie, values='Value_Weight', names='Ticker', title=f'Value-Weighted of portfolio in {date}',width=600)
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.show()
    
pie_chart_for_specific_date(df,specific_date)

In [194]:
df

,Date,PX_LAST,PX_OPEN,PX_HIGH,PX_LOW,PX_VOLUME,CUR_MKT_CAP,Ticker,Market_Value,Total_Market_Value_Day,Value_Weight,Equal_Weight
0,2014-01-02,76.27,76.04,76.500,75.900,5212460.0,134027.7199,DIS,1.022229e+07,4.859792e+07,0.210344,0.033333
1,2014-01-03,76.11,76.30,76.697,76.010,4061042.0,133746.5552,DIS,1.017945e+07,4.840901e+07,0.210280,0.033333
2,2014-01-06,75.82,76.53,76.840,75.490,6816693.0,133236.9441,DIS,1.010203e+07,4.849603e+07,0.208306,0.033333
3,2014-01-07,76.34,76.12,76.700,75.900,4511157.0,134150.7295,DIS,1.024107e+07,4.965378e+07,0.206249,0.033333
4,2014-01-08,75.22,76.13,76.280,74.810,10914858.0,132182.5763,DIS,9.942773e+06,4.934948e+07,0.201477,0.033333
...,...,...,...,...,...,...,...,...,...,...,...,...
75167,2024-08-06,54.08,52.93,55.110,52.659,16405667.0,74061.2621,CMG,4.005233e+06,5.450343e+08,0.007349,0.033333
75168,2024-08-07,53.63,54.50,55.300,53.410,11238844.0,73444.9979,CMG,3.938855e+06,5.320586e+08,0.007403,0.033333
75169,2024-08-08,54.32,54.00,54.890,53.740,9305774.0,74389.9363,CMG,4.040861e+06,5.535762e+08,0.007300,0.033333
75170,2024-08-09,55.55,54.90,56.140,54.530,11569747.0,76074.3918,CMG,4.225932e+06,5.601571e+08,0.007544,0.033333


In [191]:
# Rebalance these portfolios in the first day of the month based on the information on the last day (1-day lag)
# For example the day 2014-01-31 and 2014-02-03 now have the same Value_Weight
tickers = df["Ticker"].unique()
list_of_df_tik = []
for tik in tickers:
    df_tik = df.loc[df["Ticker"]==tik,:]
    df_tik.set_index("Date",inplace=True)
    last_day_prev_month = df_tik['Value_Weight'].resample('M').last()
    first_day_curr_month = df_tik.groupby([df_tik.index.year, df_tik.index.month]).head(1)
    last_day_prev_month_list = last_day_prev_month.to_list()
    last_day_prev_month_list.insert(0,first_day_curr_month["Value_Weight"][0])
    last_day_prev_month_list.pop(-1)
    first_day_curr_month.loc[:,"Value_Weight"] = last_day_prev_month_list
    mask = df_tik.index.isin(first_day_curr_month.index)
    df_tik.loc[mask, 'Value_Weight'] = first_day_curr_month
    df_tik.reset_index(inplace=True)
    list_of_df_tik.append(df_tik)

In [199]:
new_df = pd.concat(list_of_df_tik)
new_df = new_df.reset_index()